In [17]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [18]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11

# Reading in CSV file: 
tickers = pd.read_csv('Tickers.csv')
ticker_lst = list(tickers['Tickers'])

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)

In [ ]:
# STEP 1: FILTERING TICKERS
# Requires a Ticker list. 
def filter_stocks(tickers):
    valid_tickers, invalid_tickers = [], [];
    for ticker in tickers:
        try:
            info = yf.Ticker(ticker).fast_info;
            if (info and info.get("exchange") is not None): # filter for stocks delisted on yfinance
                if(info.get("currency") == "USD" or info.get("currency") == "CAD"): # filter for stocks that are not USD
                    valid_tickers.append(ticker);
                else:
                    invalid_tickers.append(ticker);
        except:
            invalid_tickers.append(ticker);
    return valid_tickers

ticker_lst = filter_stocks(ticker_lst)

$AGN: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


The following tickers were removed: ['AGN', 'CELG', 'MON', 'RTN']


In [35]:
data = yf.download(valid_tickers, period="2y", interval="1d")['Close']
data.index = data.index.strftime('%Y-%m-%d')
returns = data.pct_change()
returns.drop(index=returns.index[0], inplace = True)

data

[*********************100%***********************]  37 of 37 completed
/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_16191/2098675723.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change()


Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-14,148.279999,151.740005,102.010002,288.910004,60.240002,98.489998,154.130005,173.550003,37.759998,6.34,...,120.800003,131.660004,51.119999,29.120001,87.900002,177.440002,513.750000,216.949997,177.789993,43.849998
2022-11-15,150.039993,153.039993,103.139999,292.649994,60.540001,98.940002,153.889999,175.350006,37.700001,6.48,...,126.019997,131.779999,53.240002,28.930000,87.720001,177.570007,503.010010,210.660004,181.470001,43.770000
2022-11-16,148.789993,151.869995,103.610001,291.510010,59.869999,97.120003,152.570007,172.690002,37.360001,6.39,...,120.730003,131.520004,51.340000,29.070000,88.129997,173.460007,511.519989,212.270004,177.059998,42.930000
2022-11-17,150.720001,152.160004,102.559998,287.140015,59.439999,94.849998,150.639999,172.779999,37.169998,6.29,...,122.860001,132.320007,50.259998,29.010000,88.730003,175.360001,515.309998,209.820007,176.059998,42.770000
2022-11-18,151.289993,154.979996,103.870003,286.500000,59.820000,94.139999,152.500000,173.889999,37.189999,6.34,...,123.849998,132.699997,49.020000,29.170000,89.419998,175.179993,530.000000,210.580002,178.970001,43.380001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07,227.479996,200.509995,115.300003,360.709991,76.589996,210.050003,286.820007,150.979996,44.770000,3.21,...,172.910004,172.660004,118.680000,21.020000,78.739998,217.539993,605.450012,242.559998,134.149994,49.560001
2024-11-08,226.960007,199.500000,116.589996,355.529999,75.580002,208.179993,287.600006,151.679993,45.130001,3.25,...,170.910004,172.039993,121.250000,21.809999,78.910004,220.289993,615.809998,241.490005,132.440002,50.209999
2024-11-11,224.229996,174.429993,116.800003,357.299988,75.379997,206.839996,292.970001,148.960007,46.080002,3.39,...,168.289993,172.779999,125.370003,21.950001,79.279999,215.869995,625.250000,241.949997,131.610001,51.040001


In [21]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [22]:
def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [30]:
aapl = yf.Ticker('XIU')
# for exp in aapl.options:
#     aapl.option_chain(exp)

# aapl.option_chain(aapl.options[0])
len(aapl.options)

0

In [ ]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---